In [0]:
import torch, torchvision
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable as var 
import torch.nn.functional as F
from torchvision import transforms
from torch.utils import data

In [0]:
# !unzip -q "dataset.zip"

In [0]:
# training params
n_batch = 64
n_epoch = 30
n_print = 10
dropout_p = 0.2
log_interval = 1 # epochs
num_hidden_units = 50
num_classes = 10 # MNIST
max_grad_norm = 5.0

# aux training params
aux_n_batch = 64
aux_n_epoch = 300
aux_n_print = 10
aux_log_interval = 1 # epochs
aux_train_path = "dataset/train/"
aux_test_path = "dataset/test/"

In [0]:
#### Part I : Loading Your Data 
transform = transforms.Compose([ 
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),  
])

train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=transform)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=transform)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

aux_train_data = torchvision.datasets.ImageFolder(root=aux_train_path, transform=transform)
aux_train_dl = data.DataLoader(aux_train_data, batch_size=aux_n_batch, shuffle=True,  num_workers=4)

aux_test_data = torchvision.datasets.ImageFolder(root=aux_test_path, transform=transform)
aux_test_dl = data.DataLoader(aux_test_data, batch_size=aux_n_batch, shuffle=True, num_workers=4) 

In [0]:
#### Part II : Writing the Network
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN,self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.dropout_conv2 = nn.Dropout2d(dropout_p)
        self.fc1 = nn.Linear(1024, num_hidden_units)
        self.fc2 = nn.Linear(num_hidden_units, num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 1024)  # flatten
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return x   

In [45]:
#### Part III : Writing the main Training loop

mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()

optimizer = optim.Adadelta(mycnn.parameters(), lr = 1)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3)

def validate(model, data):
  # To get validation accuracy = (correct/total)*100.
    total = 0
    correct = 0
    model.eval()
    for i, (images, labels) in enumerate(data):
        images = var(images.cuda())
        x = model(images)
        value, pred = torch.max(x, 1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    return correct*100./total

for e in range(n_epoch):
    for i, (images,labels) in enumerate(train_dl): 
        mycnn.train()
        images = var(images.cuda())
        labels = var(labels.cuda())
        optimizer.zero_grad()
        pred = mycnn(images)
        loss = cec(pred,labels)
        loss.backward()
        optimizer.step()

        if (i+1) % n_print == 0:
          accuracy = float(validate(mycnn, aux_test_dl))
          print('Epoch :', e+1, 'Batch :', i+1, 'Loss :', float(loss.data), 'Accuracy :', accuracy,'%')
#     scheduler.step()
    
### Auxiliary training on data extracted from protocols

optimizer = optim.Adadelta(mycnn.parameters(), lr = 1)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)

for e in range(n_epoch, n_epoch+aux_n_epoch):
    for i, (images,labels) in enumerate(aux_train_dl):
        mycnn.train()
        images = var(images.cuda())
        labels = var(labels.cuda())
        optimizer.zero_grad()
        pred = mycnn(images)
        loss = cec(pred,labels)
        loss.backward()
        optimizer.step()
        if (i+1) % n_print == 0:
          accuracy = float(validate(mycnn, aux_test_dl))
          print('Epoch :', e+1, 'Batch :', i+1, 'Loss :', float(loss.data), 'Accuracy :', accuracy,'%')
#     scheduler.step()

Epoch : 1 Batch : 10 Loss : 2.1604323387145996 Accuracy : 16.0 %
Epoch : 1 Batch : 20 Loss : 1.5129700899124146 Accuracy : 24.0 %
Epoch : 1 Batch : 30 Loss : 0.7745266556739807 Accuracy : 30.0 %
Epoch : 1 Batch : 40 Loss : 0.7710433602333069 Accuracy : 43.0 %
Epoch : 1 Batch : 50 Loss : 0.535670280456543 Accuracy : 35.0 %
Epoch : 1 Batch : 60 Loss : 0.4667571783065796 Accuracy : 45.0 %
Epoch : 1 Batch : 70 Loss : 0.39164039492607117 Accuracy : 50.0 %
Epoch : 1 Batch : 80 Loss : 0.3712403178215027 Accuracy : 47.0 %
Epoch : 1 Batch : 90 Loss : 0.5297505855560303 Accuracy : 47.0 %
Epoch : 1 Batch : 100 Loss : 0.18805599212646484 Accuracy : 47.0 %
Epoch : 1 Batch : 110 Loss : 0.38270431756973267 Accuracy : 47.0 %
Epoch : 1 Batch : 120 Loss : 0.5592264533042908 Accuracy : 52.0 %
Epoch : 1 Batch : 130 Loss : 0.3051784634590149 Accuracy : 51.0 %
Epoch : 1 Batch : 140 Loss : 0.36128491163253784 Accuracy : 52.0 %
Epoch : 1 Batch : 150 Loss : 0.2846565842628479 Accuracy : 47.0 %
Epoch : 1 Batch 

In [0]:
### Save model
torch.save(mycnn.state_dict(), 'mnist_digits.pt')

In [0]:
### Load model
# rec_cnn = myCNN().cuda()
# model.load_state_dict(torch.load('E:/#AI&CV camp/#Project/#1Protocols/mnist_digits.pt'))
# model.eval()

<All keys matched successfully>

In [0]:
# from PIL import Image
# img = Image.open("test_6_3.jpg")

# img_t = transform(img).to(device='cuda')
# batch_t = torch.unsqueeze(img_t, 0)

In [0]:
# out = F.softmax(model(batch_t), dim=1)
# result_vec = out[0].cpu().detach().numpy()
# print (np.argmax(result_vec))